In [1]:
import pandas as pd
import datetime as dt
pd.options.display.float_format = "{:,.2f}".format

import numpy as np

import warnings
warnings.filterwarnings('ignore')

IMPORT DATA, CLEAN & CONDENSE

In [2]:
#define file names

invoice_date = input('enter invoice date: ')
date_account = 'R1' + ' ' + invoice_date
data = 'data'
condense = 'condense'
merge = 'merge'
lineitem = 'lineitem'
file_type = '.xlsx'
print(date_account)
print(date_account+file_type)

enter invoice date: June 2021
R1 June 2021
R1 June 2021.xlsx


In [3]:
#cont define file names

x = date_account + ' ' + data + file_type
y = date_account + ' ' + condense + file_type
z = date_account + ' ' + merge + file_type
w = date_account + ' ' + lineitem + file_type

print(x +'\n'+ y +'\n'+ z +'\n' + w)

R1 June 2021 data.xlsx
R1 June 2021 condense.xlsx
R1 June 2021 merge.xlsx
R1 June 2021 lineitem.xlsx


In [4]:
#import raw data, create new DF with only specific columns needed for analysis

df = pd.read_excel(x, header=None)
df2 = df.iloc[:, [2,4,5,10,11,13,16,22,45,52,75,76,78,79,80]]

In [5]:
df2.columns

Int64Index([2, 4, 5, 10, 11, 13, 16, 22, 45, 52, 75, 76, 78, 79, 80], dtype='int64')

In [6]:
#insert new column that will hold the sales order no from ref_1 column
df2.insert(6, 15, 0, True)

In [7]:
#create function to pull off sales order data

right = df2.loc[:, 16].str[1:7]

In [8]:
#set new column equal to the right 6 from ref_1

df2.loc[:, 15] = right

In [9]:
#names headers for the columns

df2.columns = ["account", "invoice_date", "invoice_no", "invoice_total", "ship_date", "tracking_no",'sales_order_no',"ref_1", "ref_2", "charge_type", "final_charge","receiving_party", "receiving_address", "city", "state", "zip" ]

In [58]:
df2.dropna(subset =['sales_order_no']).head(2)

,account,invoice_date,invoice_no,invoice_total,ship_date,tracking_no,sales_order_no,card_code,ref_1,ref_2,charge_type,final_charge,receiving_party,receiving_address,city,state,zip
1,00009A56R1,2021-06-07,1482804409,53.04,2021-06-06,9A56R1JTNDR,0,,NaN,NaN,Disbursement Fee,4.73,HUMBI,THE EA OF QINGPINGSHAN RD,MINGUANG,NaN,239400
2,00009A56R1,2021-06-07,1482804409,53.04,2021-06-06,9A56R1JTNDR,0,,NaN,NaN,Value Added Tax,48.31,HUMBI,THE EA OF QINGPINGSHAN RD,MINGUANG,NaN,239400


In [59]:
for i, row in df2.iterrows():
    df2['sales_order_no'] = df2['sales_order_no'].apply(pd.to_numeric, errors = 'coerce')

In [60]:
print(df2['sales_order_no'].dtypes)


float64


In [16]:
#inserts an empty column for cardCode

df2.insert(7, 'card_code', '')

In [17]:
#converts the column invoice_date to a datetime object, then a date object

df2['invoice_date'] = pd.to_datetime(df2['invoice_date'], errors = 'coerce')
df2['invoice_date'] = df2['invoice_date'].dt.date

#converts the column "ship date" to a datetime object, then a date object

df2['ship_date'] = pd.to_datetime(df2['ship_date'], errors = 'coerce')
df2['ship_date'] = df2['ship_date'].dt.date
  

In [18]:
#drops the second row (index)

df2 = df2.drop(df2.index[0])

In [19]:
#check data types

df2.dtypes

account               object
invoice_date          object
invoice_no            object
invoice_total        float64
ship_date             object
tracking_no           object
sales_order_no        object
card_code             object
ref_1                 object
ref_2                 object
charge_type           object
final_charge         float64
receiving_party       object
receiving_address     object
city                  object
state                 object
zip                   object
dtype: object

In [20]:
#saves new dataframe 

df2.to_excel(y, index = False)

MERGE DATA FRAMES (UPS RAW DATA WITH ORDR AND LINE ITEM TABLES)

In [48]:
#pull in ORDR sheet

df_ordr = pd.read_excel('ORDR for UPS analysis US.xlsx', sheet_name = 'ORDR_only')

In [49]:
df_ordr.dtypes

DocEntry                   int64
sales_order_no             int64
DocDate           datetime64[ns]
CardCode                  object
CardName                  object
NumAtCard                 object
dtype: object

In [22]:
df_ordr = df_ordr.astype({'sales_order_no': 'object'})

In [50]:
df_ordr.dtypes

DocEntry                   int64
sales_order_no             int64
DocDate           datetime64[ns]
CardCode                  object
CardName                  object
NumAtCard                 object
dtype: object

In [51]:
df2.dtypes

account               object
invoice_date          object
invoice_no            object
invoice_total        float64
ship_date             object
tracking_no           object
sales_order_no        object
card_code             object
ref_1                 object
ref_2                 object
charge_type           object
final_charge         float64
receiving_party       object
receiving_address     object
city                  object
state                 object
zip                   object
dtype: object

In [54]:
#pull in ORDR line items sheet 
# df_lineitems = pd.read_excel('ORDR for UPS analysis US.xlsx', sheet_name = 'ORDR_line_items')

In [65]:
#merge ORDR and UPS dataframes

df_merge = pd.merge(df2, df_ordr, on = ['sales_order_no'], how = 'left')

In [68]:
#push the new DF to excel file

df_merge.to_excel(z, index = False)

DIONO INVOICE ANALYSIS

In [69]:
#import OINV data frame

df_sales = pd.read_excel('ORDR for UPS analysis US.xlsx', sheet_name = 'OINV')

In [70]:
#analysis of revenue data

revenue = df_sales.total_sales.sum()
print('total revenue US sales')
print(format(revenue, ',.2f'))

total revenue US sales
3,529,732.81


In [71]:
df_sales.groupby(['CardCode','CardName'])[['total_sales']].sum().head()

,,total_sales
CardCode,CardName,
ALBBAB,Albee Baby Carriage (BRIXY),660.00
ALISS,Foreign Imports,"22,131.00"
AMAZON,Amazon,"1,966,282.37"
AMAZONDF,Amazon DF,"110,576.03"
ATOP,Atop Company Ltd,"9,795.50"


In [72]:
code_grp = df_sales.groupby(['CardCode'])

In [73]:
df_dioret_sales = code_grp.get_group('DIORET')

In [74]:
dioret_sales = df_dioret_sales['total_sales'].sum()
print('total revenue DIORET (online sales):')
print(format(dioret_sales, ',.2f'))

total revenue DIORET (online sales):
485,483.73


In [75]:
dioret_count = df_dioret_sales['total_sales'].count()
print('total number of orders for DIORET (online sales):')
print(format(dioret_count, ','))

total number of orders for DIORET (online sales):
2,575


In [76]:
#this needs to be adjusted to sumif on DocNum by total_sales. right now it is count line items not sales orders

print('average price per order for DIORET (online sales):')
print(format(dioret_sales/dioret_count, '.2f'))

average price per order for DIORET (online sales):
188.54


UPS DATA  ANALYSIS

In [77]:
#drop the rows where final charge = 0

df_merge = df_merge[df_merge['final_charge'] != 0]

In [78]:
#print the shape of df_merge and count unique tracking numbers
 
tracking_unique_total = df_merge.tracking_no.value_counts() 
print('dataframe shape: ' , df_merge.shape)
print('unique tracking nos: ', tracking_unique_total.count())

dataframe shape:  (9418, 22)
unique tracking nos:  2792


In [79]:
#find the new mean without the null rows. 
#aka take sum final charge divide by # of unique tracking ==> cost per carton

print('average charge overall: ', format(df_merge.final_charge.mean(), '.2f'))
print('total charges: ', format(df_merge.final_charge.sum(), ',.2f'))
print('total packages: ', format(tracking_unique_total.count(), ','))
print('average cost per package: ', format(df_merge.final_charge.sum()/tracking_unique_total.count(), '.2f'))
print('Month revenue: ', format(revenue, ',.2f'))
print('UPS freight as a percent of revenue: ', format(df_merge.final_charge.sum()/revenue, '.2%' ))

average charge overall:  5.81
total charges:  54,696.46
total packages:  2,792
average cost per package:  19.59
Month revenue:  3,529,732.81
UPS freight as a percent of revenue:  1.55%


UPS INVOICE SUMMARY

In [80]:
#sum final charge by invoice number and date
print('total charges: ', format(df_merge.final_charge.sum(), ',.2f'))
print('total packages: ', format(tracking_unique_total.count(), ','))
print('average cost per package: ', format(df_merge.final_charge.sum()/tracking_unique_total.count(), '.2f'))


total charges:  54,696.46
total packages:  2,792
average cost per package:  19.59


In [81]:
cardcode = df_merge["CardCode"]
cardcode.head(10)

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
7        NaN
8        NaN
9     DIORET
10    DIORET
Name: CardCode, dtype: object

In [82]:
#dataframe groupby object
df_merge.groupby(['CardCode'])


In [83]:
#dataframe groupby object
code_grp = df_merge.groupby(['CardCode'])

#create dataframe of the code group = DIORET
df_dioret = code_grp.get_group('DIORET')

In [84]:
sum_finalcharge = df_merge[['final_charge']].sum()
invoice_by_date = df_merge.groupby(['invoice_date', 'invoice_no'])[['final_charge']].sum()
dioret_total_sum = df_dioret['final_charge'].sum()

print('invoice data')
print(invoice_by_date)
print(sum_finalcharge)
print("online sales shipping charges total: " , dioret_total_sum)

invoice data
                              final_charge
invoice_date invoice_no                   
2021-06-05   0000009A56R1231     15,203.66
2021-06-07   1482804409              53.04
2021-06-12   0000009A56R1241     20,882.24
2021-06-19   0000009A56R1251      5,762.64
2021-06-26   0000009A56R1261     12,794.88
final_charge   54,696.46
dtype: float64
online sales shipping charges total:  39949.94


In [85]:
# df_merge['CardCode'].value_counts()

DIONO ONLINE SALES FREIGHT SUMMARY

In [86]:
#analysis on the DIORET dataframe

dioret_total_tracking = df_dioret['tracking_no'].value_counts().count()
dioret_total_sum = df_dioret['final_charge'].sum()
print('total DIORET tracking no:', format(dioret_total_tracking, ','))
print('total DIORET sum final charge:', format(dioret_total_sum, ',.2f'))
print('average cost DIORET shipment: ', format(dioret_total_sum/dioret_total_tracking, '.2f'))

total DIORET tracking no: 2,106
total DIORET sum final charge: 39,949.94
average cost DIORET shipment:  18.97


In [87]:
print('UPS freight charges as a percent of sales- DIORET')
print('total UPS charges for DIORET: ', format(dioret_total_sum, ','))
print('total DIORET revenue: ', format(dioret_sales, ','))
print(format(dioret_total_sum/dioret_sales, '.2%'))


UPS freight charges as a percent of sales- DIORET
total UPS charges for DIORET:  39,949.94
total DIORET revenue:  485,483.73
8.23%


In [88]:
#sums final charge by customer
print('SUM OF CHARGES BY CUSTOMER')
print(df_merge.groupby(['CardCode','CardName'])[['final_charge']].sum())

SUM OF CHARGES BY CUSTOMER
                                                    final_charge
CardCode     CardName                                           
AMAZONDF     Amazon DF                                    279.72
BABGRA       Baby Grand                (BRIXY)            174.66
BRIBAB       Bringing Up Baby                               8.92
CALSPANC     Calspan Corp                                  21.22
CAMB01       Cambria Recall 2020 01                     1,011.60
CHEBAB       CHEEKY BABY                                   68.79
CHIALA       CHILDREN'S HOSPITAL OF ALABAMA                78.28
COOSER-DS    Coordinated Claim Services drop ship          28.17
CPST         CPST                                         127.89
DIORET       B2C Credit Card                           39,949.94
FACBOK       Facebook                                      23.05
IBIGRO       IBIYA GROUP                                   76.71
JOHCHI       John Hopkins All Childrens                    54.8

MERGE RDR1 LINE ITEM DETAIL WITH UPS

In [89]:
#merge line item table with UPS data. drop NA value rows

df_detail = pd.merge(df_lineitems, df2, on = ['sales_order_no'], how = 'left')
df_detail = df_detail.dropna()
# df_detail.head(2)

In [90]:
#push line item merged table to excel doc
df_detail.to_excel(w, index = False )